# Using rosetta-conf to generate native configuration

## Starting things up

Run:

    make start-dev-containers

Now, let's export some environment variables we need:

In [1]:
export USER_CERT=../../tests/certs/test_user_curl.pem
export BASE_URL=https://ntc-rosetta-conf:8443

## Configuration

First we need to configure the platform of the device, this configuration is part of the model.

In [2]:
cat 4_translate/configuration.json

{
    "ntc-rosetta-conf:device": {
        "config": {
            "platform": "ios"
        }
    }
}


In [3]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @4_translate/configuration.json \
    $BASE_URL/restconf/data/

In [4]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-commit

{
    "status": "OK",
    "conf-changed": true
}

In [5]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf_running/data/ntc-rosetta-conf:device

{
    "ntc-rosetta-conf:device": {
        "config": {
            "platform": "ntc-rosetta-conf:ios"
        }
    }
}

## Translating

The first thing we can do is translate the data into native format. Let's start by adding a couple of interfaces:

In [6]:
cat 4_translate/add_interface_eth0.json

{
    "openconfig-interfaces:interface": {
        "name": "eth0",
        "config": {
            "name": "eth0",
            "description": "an interface description",
            "type": "iana-if-type:ethernetCsmacd"
        }
    }
}


In [7]:
cat 4_translate/add_interface_eth1.json

{
    "openconfig-interfaces:interface": {
        "name": "eth1",
        "config": {
            "name": "eth1",
            "description": "another interface",
            "type": "iana-if-type:ethernetCsmacd"
        }
    }
}


In [8]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @4_translate/add_interface_eth0.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

In [9]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @4_translate/add_interface_eth1.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

In [10]:
# candidate database
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

{
    "openconfig-interfaces:interfaces": {
        "interface": [
            {
                "name": "eth0",
                "config": {
                    "name": "eth0",
                    "description": "an interface description",
                    "type": "iana-if-type:ethernetCsmacd"
                }
            },
            {
                "name": "eth1",
                "config": {
                    "name": "eth1",
                    "description": "another interface",
                    "type": "iana-if-type:ethernetCsmacd"
                }
            }
        ]
    }
}

In [11]:
# running database
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf_running/data/openconfig-interfaces:interfaces

{
    "openconfig-interfaces:interfaces": {
        "interface": []
    }
}

Now we can translate either the "candidate" or "running" databases into native configuration:

In [12]:
cat 4_translate/translate_candidate.json

{
    "ntc-rosetta-conf:input": {
        "database": "candidate"
    }
}


In [13]:
curl -s --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @4_translate/translate_candidate.json \
    $BASE_URL/restconf/operations/ntc-rosetta-conf:translate | jq -r ".native"

interface eth0
   description an interface description
   exit
!
interface eth1
   description another interface
   exit
!



In [14]:
# running was empty so no data there
curl -s --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @4_translate/translate_running.json \
    $BASE_URL/restconf/operations/ntc-rosetta-conf:translate | jq -r ".native"

In [15]:
# now we are happy with it we can commit the configuration
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-commit

{
    "status": "OK",
    "conf-changed": true
}

## More changes

We can now apply more changes to the candidate database and keep comparing the candidate and running databases of the device using their native representation:

In [16]:
cat 4_translate/change_interface_eth0.json

{
    "openconfig-interfaces:description": "a changed description"
}


In [17]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X PUT \
    -d @4_translate/change_interface_eth0.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces/interface=eth0/config/description

In [18]:
curl -s --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @4_translate/translate_candidate.json \
    $BASE_URL/restconf/operations/ntc-rosetta-conf:translate | jq -r ".native"

interface eth0
   description a changed description
   exit
!
interface eth1
   description another interface
   exit
!



In [19]:
curl -s --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @4_translate/translate_running.json \
    $BASE_URL/restconf/operations/ntc-rosetta-conf:translate | jq -r ".native"

interface eth0
   description an interface description
   exit
!
interface eth1
   description another interface
   exit
!



At this point you could grab both "native" results and diff them:

In [20]:
curl -s --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @4_translate/translate_candidate.json \
    $BASE_URL/restconf/operations/ntc-rosetta-conf:translate | jq -r ".native" > /tmp/candidate
curl -s --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @4_translate/translate_running.json \
    $BASE_URL/restconf/operations/ntc-rosetta-conf:translate | jq -r ".native" > /tmp/running
diff -W 100 --side-by-side /tmp/candidate /tmp/running || echo -n

interface eth0					interface eth0
   description a changed description	      |	   description an interface description
   exit						   exit
!						!
interface eth1					interface eth1
   description another interface		   description another interface
   exit						   exit
!						!



## Merging

Alterntatively, you can call the merge endpoint and get a list of commands that will make the configurations converge:

In [21]:
curl -s --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @4_translate/merge.json \
    $BASE_URL/restconf/operations/ntc-rosetta-conf:merge | jq -r ".native"

interface eth0
   description a changed description
   exit
!



In [22]:
# ignore me, this deletes the data so the notebook can be rerun
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-reset
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X PUT \
    -d @../../tests/data/interfaces_empty.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X DELETE \
    $BASE_URL/restconf/data/ntc-rosetta-conf:device
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-commit

{
    "status": "OK"
}{
    "status": "OK",
    "conf-changed": true
}